<a href="https://colab.research.google.com/github/SabinSapota/Blooging/blob/main/encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**Importance of Level encoding,Onehot encoder and factorisation for dealing categoricals values**


 for this I use melbourne housing dataset

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [50]:
# laod the data set
df=pd.read_csv('/content/drive/MyDrive/spam/melb_data.csv')
df.head(2)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0


In [8]:
df.shape

(13580, 21)

In [9]:
# columns with missing value
my_cols=[col for col in df.columns if df[col].isna().any()]
my_cols

['Car', 'BuildingArea', 'YearBuilt', 'CouncilArea']

In [55]:
# for this case, I will drop the missing columns
df=df.drop(my_cols,axis=1)

In [11]:
# shape after dropping na value
df.shape

(13580, 17)

In [56]:
# input features(independent feature)
X=df.drop('Price',axis=1)
# dependentdent feature
y=df['Price']

In [57]:
# here i am choosing columns which have low cardinality(meaning uniqueness) less than 10 and are string data types
low_car=[cname for cname in X.columns if X[cname].nunique()<10 and X[cname].dtype=='object']
low_car

['Type', 'Method', 'Regionname']

In [58]:
# now choosing numeric columns only
num_col=[cname for cname in X.columns if X[cname].dtype in ['int64','float64']]
num_col

['Rooms',
 'Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Landsize',
 'Lattitude',
 'Longtitude',
 'Propertycount']

In [59]:
# combining above two columns
my_col=low_car+num_col

X=X[my_col]

In [60]:
# my final data set
X.head()


,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
0,h,S,Northern Metropolitan,2,2.5,3067.0,2.0,1.0,202.0,-37.7996,144.9984,4019.0
1,h,S,Northern Metropolitan,2,2.5,3067.0,2.0,1.0,156.0,-37.8079,144.9934,4019.0
2,h,SP,Northern Metropolitan,3,2.5,3067.0,3.0,2.0,134.0,-37.8093,144.9944,4019.0
3,h,PI,Northern Metropolitan,3,2.5,3067.0,3.0,2.0,94.0,-37.7969,144.9969,4019.0
4,h,VB,Northern Metropolitan,4,2.5,3067.0,3.0,1.0,120.0,-37.8072,144.9941,4019.0


In [43]:
# getting categogicals columns
[cname for cname in X.columns if X[cname].dtype=='object']

['Type', 'Method', 'Regionname']

Now, I  used MAE(mean absulute error) to evalute the regression problem's accuracy since this is a regression problems

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches by calculating MAE
def score_dataset(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return mean_absolute_error(y_test, preds)

First we calculate MAE(Mean absolute error) by dropping categoricals columns

In [48]:

#Divide data into training and validation subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_test = X_test.select_dtypes(exclude=['object'])


print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_test, y_train, y_test))

MAE from Approach 1 (Drop categorical variables):
175703.48185157913


Label Encoding

In [17]:
# extracting columns with string
x=[cname for cname in X.columns if X[cname].dtype=='object']
x


['Type', 'Method', 'Regionname']

In [68]:
X=X[my_col]

In [18]:
from sklearn.preprocessing import LabelEncoder
my_model=LabelEncoder()
for i in x:
  X[i]=my_model.fit_transform(df[i])

In [19]:
#Divide data into training and validation subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

In [23]:
print('MAE value with label Enocding:')
score_dataset(X_train,X_test,y_train,y_test)

MAE value with label Enocding:


165936.40548390493

**factorise method**

In [62]:
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()


In [63]:

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)
score_dataset(X_train,X_test,y_train,y_test)


167929.79475313835

**OneHot Encoding**

In [ ]:
from sklearn.preprocessing import  OneHotEncoder
ohe=OneHotEncoder()

In [ ]:
for i in x:
  ##reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object
  X[i] = ohe.fit_transform(X[i].values.reshape(-1,1)).toarray()

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)
score_dataset(X_train,X_test,y_train,y_test)

168502.60937745858

Conclusion,
  we can see after  reducing categoricals columns to respective label encoding,one hot encoding and factorising columns we have reduced the MAE error bewteen actual value and predicted value.
  Lower the MAE value , better the model.